In [1]:
!pip install --upgrade setuptools==65.5.0
!pip install --quiet stable_baselines3
!pip install --quiet import_ipynb
!git clone https://github.com/gmshroff/aiagentarch.git
!pip install pyglet==1.5.27
%cd aiagentarch

fatal: destination path 'aiagentarch' already exists and is not an empty directory.
/Users/joykirat/Downloads/sem8/MTL/Hebbian_MetaRL/Project/aiagentarch


In [2]:
import gym
from gym import spaces
from gym import Env
import random
import numpy as np
from threading import Thread
import threading
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv,VecFrameStack
from stable_baselines3.common.monitor import Monitor as Mon
from stable_baselines3.common.vec_env import StackedObservations
import random
import torch.nn as nn

In [3]:
import import_ipynb
from aiagentbase import AIAgent,Controller,Memory,Perception,Actor
from cartpole_tasks import CartPoleEnv
from gym.spaces import Discrete, Box
import torch
from typing import List, Any


importing Jupyter notebook from aiagentbase.ipynb
importing Jupyter notebook from cartpole_tasks.ipynb


In [4]:
import multiprocess as mp

### Agent-based RL in Simple Worlds

In [5]:
# environment = 'CartPole-v1'
# hebb_rule = 'ABCD_lr' # A, AD, AD_lr, ABC, ABC_lr, ABCD, ABCD_lr, ABCD_lr_D_out, ABCD_lr_D_in_and_out
# popsize = 10 # Population size # 200
# learning_rate = 0.2 # Learning rate
# decay = 0.995 # Learning rate decay
# SIGMA = 0.1 # Modulates the amount of noise used to populate each new generation
# init_weights = 'uni' # The distribution used to sample random weights from at each episode: uni, normal, default, xa_uni, sparse, ka_uni or coevolve to co-evolve the intial weights
# print_every = 1 # Print and save every N steps
# generations = 5 # Number of generations that the ES will run # 50
# threads = -1 # Number of threads used to run evolution in parallel: -1 uses all threads available
# folder = 'heb_coeffs' # Folder to store the evolved Hebbian coefficients
# distribution = 'normal' # Sampling distribution for initialize the Hebbian coefficients: normal, uniform


In [21]:
args = {
    'environment' : 'CartPole-v1',
    'hebb_rule' : 'ABCD_lr', # A, AD, AD_lr, ABC, ABC_lr, ABCD, ABCD_lr, ABCD_lr_D_out, ABCD_lr_D_in_and_out
    'popsize' : 10, # Population size # 200
    'learning_rate' : 0.2, # Learning rate
    'decay' : 0.995, # Learning rate decay
    'sigma' : 0.1, # Modulates the amount of noise used to populate each new generation
    'init_weights' :'uni', # The distribution used to sample random weights from at each episode: uni, normal, default, xa_uni, sparse, ka_uni or coevolve to co-evolve the intial weights
    'print_every' : 1, # Print and save every N steps
    'generations' : 5, # Number of generations that the ES will run # 50
    'threads' : -1, # Number of threads used to run evolution in parallel: -1 uses all threads available
    'folder' : 'heb_coeffs', # Folder to store the evolved Hebbian coefficients
    'distribution' : 'normal', # Sampling distribution for initialize the Hebbian coefficients: normal, uniform
}

In [23]:
num_threads = mp.cpu_count() if args['threads'] == -1 else args['threads']
update_factor = args['learning_rate'] / (args['popsize'] * args['sigma'])
distribution = args['distribution']                      

# The number of hebbian coefficients per synapse
if args['hebb_rule'] == 'ABCD_lr':                                           
    coefficients_per_synapse = 5
else:
    raise ValueError('The provided Hebbian rule is not valid')
    

# Look up observation and action space dimension
env = CartPoleEnv()

if len(env.observation_space.shape) == 1:   # State-based environment 
    pixel_env = False
    input_dim = env.observation_space.shape[0]
elif isinstance(env.observation_space, Discrete):
    pixel_env = False
    input_dim = env.observation_space.n
else:
    raise ValueError('Observation space not supported')

if isinstance(env.action_space, Box):
    action_dim = env.action_space.shape[0]
elif isinstance(env.action_space, Discrete):
    action_dim = env.action_space.n
else:
    raise ValueError('Action space not supported')
            
# Initialize the values of hebbian coefficients and CNN parameters or initial weights of co-evolving initial weights   
            
# State-vector environments (MLP)            
plastic_weights = (128*input_dim) + (64*128) + (action_dim*64)                                            #  Hebbian coefficients:  MLP x coefficients_per_synapse :plastic_weights x coefficients_per_synapse

# Co-evolution of initial weights                

# Random initial weights
if distribution == 'uniform': 
    coeffs = np.random.uniform(-1,1,(plastic_weights, coefficients_per_synapse)) 
elif distribution == 'normal':
    coeffs = torch.randn(plastic_weights, coefficients_per_synapse).detach().numpy().squeeze() 

In [24]:
env.length

0.5

In [27]:
class MLP_heb(nn.Module):
    "MLP, no bias"
    def __init__(self, input_space, action_space):
        super(MLP_heb, self).__init__()

        self.fc1 = nn.Linear(input_space, 128, bias=False)
        self.fc2 = nn.Linear(128, 64, bias=False)
        self.fc3 = nn.Linear(64, action_space, bias=False)

    def forward(self, ob):
        import pickle
        with open('ob.pkl', 'wb') as f:
            pickle.dump(ob, f)
        new_ob = ob[0][0]

        if(type(new_ob) == np.float32):
            new_ob = ob[0]
        state = torch.as_tensor(new_ob).float().detach()
        # print(state)
        x1 = torch.tanh(self.fc1(state))   
        x2 = torch.tanh(self.fc2(x1))
        o = self.fc3(x2)  
         
        return state, x1, x2, o    

In [82]:
def hebbian_update_ABCD_lr_D_in(heb_coeffs, weights1_2, weights2_3, weights3_4, o0, o1, o2, o3):
       
        heb_offset = 0
        ## Layer 1         
        for i in range(weights1_2.shape[1]): 
            for j in range(weights1_2.shape[0]): 
                idx = (weights1_2.shape[0]-1)*i + i + j
                weights1_2[:,i][j] += heb_coeffs[idx][3] * ( heb_coeffs[idx][0] * o0[i] * o1[j]
                                                           + heb_coeffs[idx][1] * o0[i] 
                                                           + heb_coeffs[idx][2]         * o1[j]  + heb_coeffs[idx][4])

        heb_offset += weights1_2.shape[1] * weights1_2.shape[0]
        # Layer 2
        for i in range(weights2_3.shape[1]): 
            for j in range(weights2_3.shape[0]):
                idx = heb_offset + (weights2_3.shape[0]-1)*i + i+j
                weights2_3[:,i][j] += heb_coeffs[idx][3] * ( heb_coeffs[idx][0] * o1[i] * o2[j]
                                                           + heb_coeffs[idx][1] * o1[i] 
                                                           + heb_coeffs[idx][2]         * o2[j]  + heb_coeffs[idx][4])

        heb_offset += weights2_3.shape[1] * weights2_3.shape[0]
        # Layer 3
        for i in range(weights3_4.shape[1]): 
            for j in range(weights3_4.shape[0]): 
                idx = heb_offset + (weights3_4.shape[0]-1)*i + i+j 
                weights3_4[:,i][j] += heb_coeffs[idx][3] * ( heb_coeffs[idx][0] * o2[i] * o3[j]
                                                           + heb_coeffs[idx][1] * o2[i] 
                                                           + heb_coeffs[idx][2]         * o3[j]  + heb_coeffs[idx][4])
                
        return weights1_2, weights2_3, weights3_4

In [117]:
class CartWorldMeta():
    def __init__(self,env=env,agent=None):
        self.env=env
        self.test_episodes=[]
        self.world_over=False
        self.agent = agent
    def stop(self):
        self.world_over=True
    def run_all(self,perf,n_episodes=10,total_tasks=None,replay=None):
        if total_tasks is None:
            for g in perf:
                for l in perf[g]:
                    self.env.length,self.env.gravity=l,g
                    self.env.polemass_length = self.env.masspole * self.env.length
                    self.run(n_episodes=n_episodes)
            return []
        elif replay is None:
            replay=[]
            for task in range(total_tasks):
                env.length=random.choice(list(perf[10].keys()))
                env.gravity=random.choice(list(perf.keys()))
                replay+=[(env.length,env.gravity)]
                env.polemass_length = env.masspole * env.length
                print(f'Task:{task},length:{env.length},gravity:{env.gravity}')
                self.run(n_episodes=n_episodes)
            return replay
        else:
            for l,g in replay:
                env.length=l
                env.gravity=g
                env.polemass_length = env.masspole * env.length
                print(f'length:{env.length},gravity:{env.gravity}')
                self.run(n_episodes=n_episodes)
            return replay
        
    def get_population(self,coevolved_param = False): 
        
        population = []
            
        if coevolved_param == False:
            for i in range( int(self.agent.args['popsize']/2) ):
                x = []
                x2 = []
                for w in coeffs:
                    j = np.random.randn(*w.shape)             # j: (coefficients_per_synapse, 1) eg. (5,1)
                    x.append(j)                                                   # x: (coefficients_per_synapse, number of synapses) eg. (92690, 5)
                    x2.append(-j) 
                population.append(x)                                              # population : (population size, coefficients_per_synapse, number of synapses), eg. (10, 92690, 5)
                population.append(x2)
                
        return np.array(population).astype(np.float32)
    def worker_process_hebb(self,arg):
        get_reward_func, hebb_rule,  environment,  init_weights, coeffs, episode_maxlen = arg
        
        wp = np.array(coeffs)
        decay = - 0.01 * np.mean(wp**2)
        r = get_reward_func( hebb_rule,  environment,  init_weights, coeffs,episode_maxlen) + decay
        
        return r
    
    def _get_params_try(self, w, p):

        param_try = []
        for index, i in enumerate(p):
            jittered = self.agent.args['sigma'] * i
            param_try.append(w[index] + jittered)
        param_try = np.array(param_try).astype(np.float32)
        
        return param_try
    

    def get_worker_args(self,pool , population, episode_maxlen):

        rewards = []
        for p in population:
            heb_coeffs_try = np.array(self._get_params_try(coeffs, p))
            rewards.append(self.fitness( self.agent.args['hebb_rule'], self.agent.args['environment'],  self.agent.args['init_weights'], heb_coeffs_try))
        
        rewards = np.array(rewards).astype(np.float32)

        return rewards
    
    def fitness(self,hebb_rule : str, environment : str, init_weights = 'uni' , *evolved_parameters: List[np.array], episode_maxlen = 100) -> float:
        def weights_init(m):
            if isinstance(m, torch.nn.Linear):
                if init_weights == 'xa_uni':  
                    torch.nn.init.xavier_uniform(m.weight.data, 0.3)
                elif init_weights == 'sparse':  
                    torch.nn.init.sparse_(m.weight.data, 0.8)
                elif init_weights == 'uni':  
                    torch.nn.init.uniform_(m.weight.data, -0.1, 0.1)
                elif init_weights == 'normal':  
                    torch.nn.init.normal_(m.weight.data, 0, 0.024)
                elif init_weights == 'ka_uni':  
                    torch.nn.init.kaiming_uniform_(m.weight.data, 3)
                elif init_weights == 'uni_big':
                    torch.nn.init.uniform_(m.weight.data, -1, 1)
                elif init_weights == 'xa_uni_big':
                    torch.nn.init.xavier_uniform(m.weight.data)
                elif init_weights == 'ones':
                    torch.nn.init.ones_(m.weight.data)
                elif init_weights == 'zeros':
                    torch.nn.init.zeros_(m.weight.data)
                elif init_weights == 'default':
                    pass

        # Unpack evolved parameters
        try: 
            hebb_coeffs, initial_weights_co = evolved_parameters
        except: 
            hebb_coeffs = evolved_parameters[0]
        with torch.no_grad():
            
            # decide whether to use initiase a new env or use the old env

            if len(env.observation_space.shape) == 1:   
                input_dim = env.observation_space.shape[0]
            elif len(env.observation_space.shape) == 0:   
                input_dim = env.observation_space.n
            
            

                
            # Determine action space dimension
            if isinstance(env.action_space, Discrete):
                action_dim = env.action_space.n
            else:
                raise ValueError('Only Box and Discrete action spaces supported')
            
            p = MLP_heb(input_dim, action_dim)
            p.apply(weights_init)
            p = p.float()
            weights1_2, weights2_3, weights3_4 = list(p.parameters())
            weights1_2 = weights1_2.detach().numpy()
            weights2_3 = weights2_3.detach().numpy()
            weights3_4 = weights3_4.detach().numpy()

            observation = self.env.reset()
            self.agent.begin()

            rew_eq = 0
            from tqdm import tqdm
            # for t in (range(episode_maxlen)):
            while True:
                o0, o1, o2, o3 = p([observation])
                o0 = o0.numpy()
                o1 = o1.numpy()
                o2 = o2.numpy()
                action = self.agent.act(o3)
                o3 = o3.numpy()
                observation, reward, done, info = self.env.step(action)
                # print("rew : ", reward)
                rew_eq += reward
                self.agent.reward((reward, done, info))

                if done:
                    break
                weights1_2, weights2_3, weights3_4 = hebbian_update_ABCD_lr_D_in(hebb_coeffs, weights1_2, weights2_3, weights3_4, o0, o1, o2, o3)
                
                (a, b, c) = (0, 1, 2)
                list(p.parameters())[a].data /= list(p.parameters())[a].__abs__().max()
                list(p.parameters())[b].data /= list(p.parameters())[b].__abs__().max()
                list(p.parameters())[c].data /= list(p.parameters())[c].__abs__().max()
            env.close()
        return rew_eq


    def run(self,n_episodes=10,episode_maxlen=1000):
        self.agent.observation_space=env.observation_space
        if 'training' not in self.agent.__dict__: self.agent.training=False
        if self.agent.training: testing=False 
        else: testing=True
        if self.agent.training: print('Starting Training time: ',self.agent.time)
        for episode in range(n_episodes):
            generation_rewards = []
            pool = mp.Pool(num_threads) if num_threads > 1 else None
            for generation in range(self.agent.args['generations']):
                population = self.get_population()
                rewards = self.get_worker_args(pool, population, episode_maxlen)
                print(rewards)
                self.agent._update_coeffs(rewards, population)
#                 self.agent.reward((rewards, population))
                


            # print('CartAgent','starting episode')
            # state=self.env.reset()
            # self.agent.begin()
            # # print(agent.time)#,agent.ep)
            # for t in range(episode_maxlen):
            #     # print(f'length:{env.length},gravity:{env.gravity}')
            #     # env.render(mode='rgb_array')
            #     action=self.agent.act(state)
            #     # print(episode,t,'Action: ', action)
            #     state, reward, done, info = env.step(action)
            #     self.agent.reward((reward,done,info))
            #     # print(episode,t,'Reward sent: ', reward)
            #     if done:
            #         break
            if self.world_over:break
            if not self.agent.training: self.test_episodes+=[episode]
            if not self.agent.training and not testing: 
                print('Training Over at time: ',self.agent.time)
                testing=True
                self.world_over=True
        print('Testing Done time: ', self.agent.time, ' Reward: ', self.agent.avg_rew())
        return self.agent.avg_rew()

In [118]:
class HebbianAgent(AIAgent):
    def __init__(self, args):
        super().__init__()
        self.actor=self.Actor(parent=self)
        self.tot_rew=0
        # self.mlp_heb = MLP_heb(4, 2)
        self.rewL=[]
        self.args = args
        
    class Actor(Actor):
        def __init__(self,parent): 
            super().__init__(parent=parent)

        def call_model(self,observation):
            action = np.argmax(observation).numpy()
            return action
            
        def compute_reward(self,reward):
            return reward[0]
    
    def reward(self,rew):
#         print(rew)
#         # learning_rate = self.args["learning_rate"]
#         # sigma = self.args["sigma"]
#         # decay = self.args["decay"]
# #         print("Initial rew ", rew, type(rew))
#         rewards = rew[0]
#         population = rew[1]
        
#         rewards = self.compute_centered_ranks(rewards)
#         std = rewards.std()
#         if std == 0:
#             raise ValueError('Variance should not be zero')
        
#         rewards = (rewards - rewards.mean()) / std
# #         print("rewards :", rewards)
#         for index, c in enumerate(coeffs):
#             layer_population = np.array([p[index] for p in population])
#             update_factor = self.agent.args['learning_rate'] / (self.agent.args['popsize'] * self.agent.args['sigma'])
                      
#             coeffs[index] = c + update_factor * np.dot(layer_population.T, rewards).T 

#         if(self.agent.args['learning_rate'] > 0.001):
#             self.agent.args['learning_rate'] *= self.args["decay"]

#         if(self.args["sigma"] > 0.001):
#             self.args["sigma"] *= 0.999
  
#         ##Augmenting AIAgent
        self.tot_rew+=rew[0]
# #         print(rew[0].mean())


        return super().reward(rew)
    
    def _update_coeffs(self, rewards, population):
            rewards = self.compute_centered_ranks(rewards)
            std = rewards.std()
            if std == 0:
                raise ValueError('Variance should not be zero')
            
            rewards = (rewards - rewards.mean()) / std
    #         print("rewards :", rewards)
            for index, c in enumerate(coeffs):
                layer_population = np.array([p[index] for p in population])
                update_factor = self.args['learning_rate'] / (self.args['popsize'] * self.args['sigma'])
                        
                coeffs[index] = c + update_factor * np.dot(layer_population.T, rewards).T 

            if(self.args['learning_rate'] > 0.001):
                self.args['learning_rate'] *= self.args["decay"]

            if(self.args["sigma"] > 0.001):
                self.args["sigma"] *= 0.999

    def compute_ranks(self,x):
        """
        Returns rank as a vector of len(x) with integers from 0 to len(x)
        """
        assert x.ndim == 1
        ranks = np.empty(len(x), dtype=int)
        ranks[x.argsort()] = np.arange(len(x))
        return ranks
    def compute_centered_ranks(self,x):
        """
        Maps x to [-0.5, 0.5] and returns the rank
        """
        y = self.compute_ranks(x.ravel()).reshape(x.shape).astype(np.float32)
        y /= (x.size - 1)
        y -= .5
        return y
    def begin(self):
        ##Augmenting AIAgent
        self.rewL+=[self.tot_rew]
        super().begin()
    def avg_rew(self):
        return sum(self.rewL)/len(self.rewL)

In [119]:
class RandomAIAgent(AIAgent):
    def __init__(self,action_space):
        super().__init__()
        self.actor=self.Actor(parent=self)
        self.action_space=action_space
        self.tot_rew=0
        self.rewL=[]
        
    class Actor(Actor):
        def __init__(self,parent): 
            super().__init__(parent=parent)
        def call_model(self,state):
        ##Overriding AIAgent.Model
            # print(self.parent.action_space.sample())
            action = self.parent.action_space.sample()
            return action
        def compute_reward(self,reward):
            return reward[0]
    
    def reward(self,rew):
        ##Augmenting AIAgent
        self.tot_rew+=rew[0]
        print("reward : ", rew, type(rew))
        return super().reward(rew)
    def begin(self):
        ##Augmenting AIAgent
        self.rewL+=[self.tot_rew]
        super().begin()
    def avg_rew(self):
        return sum(self.rewL)/len(self.rewL)

In [120]:
import pickle
with open('./perf.pickle','rb') as f: perf=pickle.load(f)

In [121]:
agent=HebbianAgent(args)


In [122]:
env = CartPoleEnv()


In [123]:
agent.training=True
agent.debug=False
agent.use_memory=True
agent.limit_memory=True
agent.memory.limit_perceptual=2
agent.memory.limit_sar=4

In [124]:
world=CartWorldMeta(env=env, agent=agent)

In [125]:
r=world.run_all(perf,total_tasks=20,replay=None)

Task:0,length:4.25,gravity:140
Starting Training time:  0
[35. 30. 28. 55. 42. 41. 48. 36. 52. 38.]
[32. 27. 57. 21. 18. 19. 22. 23. 28. 44.]
[49. 40. 31. 36. 30. 67. 28. 29. 21. 34.]
[20. 22. 36. 20. 19. 24. 22. 36. 20. 35.]
[33. 27. 35. 29. 35. 46. 64. 30. 30. 32.]
[31. 25. 31. 29. 48. 21. 39. 26. 19. 24.]
[32. 26. 23. 33. 33. 36. 37. 19. 25. 20.]
[20. 27. 22. 20. 43. 25. 37. 27. 39. 20.]


KeyboardInterrupt: 

In [ ]:
agent.avg_rew()/len(agent.ep)

In [ ]:
# agent.memory.perceptual_memory

In [ ]:
with open('./replay.pickle','wb') as f: pickle.dump(r,f)

### Training an AI Agent's Model using Generic RL Agent

In [ ]:
from threading import Thread
import threading
import sys

In [ ]:
from queue import Queue

In [ ]:
from aiagentbase import RLAgent

In [ ]:
training_steps=1000000

In [ ]:
agent=RLAgent(algoclass=PPO,monclass=Mon,action_space=env.action_space,observation_space=env.observation_space,
              verbose=0,metarl=True,win=10,soclass=StackedObservations)

In [ ]:
agent.debug=False
agent.use_memory=True
agent.training=True

In [ ]:
agent.rewL=[]
agent.tot_rew=0

In [ ]:
agent.start(training_steps=training_steps)

In [ ]:
world=CartWorldMeta(env=env)

In [ ]:
# worldthread=Thread(name='world',target=world.run,args=(agent,2000,200))

In [ ]:
# worldthread.start()

In [ ]:
# len(agent.logL)

In [ ]:
# agent.time

In [ ]:
# agent.memory.sar_memory[10011]

In [ ]:
with open('./replay.pickle','rb') as f: replay=pickle.load(f)

In [ ]:
_=world.run_all(agent,perf,n_episodes=50,total_tasks=20,replay=replay)

In [ ]:
# Without metarl training
agent.avg_rew()/len(agent.ep)

In [ ]:
# With metarl training
agent.avg_rew()/len(agent.ep)

In [ ]:
agent.training=False

In [ ]:
_=world.run_all(agent,perf,n_episodes=50,total_tasks=20,replay=replay)

In [ ]:
# Without metarl testing
agent.avg_rew()/len(agent.ep)

In [ ]:
# With metarl testing
agent.avg_rew()/len(agent.ep)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# testing_len=len([agent.rewL[t] for t in world.test_episodes])

In [ ]:
# testing_len

In [ ]:
# agent.rewL

In [ ]:
# print(np.gradient(agent.rewL).mean())

In [ ]:
# Without metarl 
plt.plot(np.gradient(agent.rewL))

In [ ]:
# With metarl 
plt.plot(np.gradient(agent.rewL))

In [ ]:
episodes = 50
rewL=[]
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    steps=0
    while not done and steps<=200:
        env.render()
        action=agent.act(state)
        # action,_ = agent.model.predict(state)
        state, reward, done, info = env.step(action)
        score+=reward
        steps+=1
    # print('Episode:{} Score:{}'.format(episode, score))
    rewL+=[score]
env.close()

In [ ]:
print(np.array(rewL).mean())

In [ ]:
plt.plot(rewL)

###### 